In [1]:
import os

try:
    # Running as normal Python script inside src/
    this_file = os.path.abspath(__file__)
    src_root = os.path.dirname(this_file)                        # EMOTION-PRED/src
    project_root = os.path.dirname(src_root)                    # EMOTION-PRED/
except NameError:
    # Running inside Jupyter (likely src/notebooks or src/)
    cwd = os.getcwd()

    # If running inside src/notebooks → go up one level
    if cwd.endswith("notebooks"):
        src_root = os.path.abspath(os.path.join(cwd, ".."))
        project_root = os.path.dirname(src_root)
    else:
        # Running from project root directly
        project_root = cwd
        src_root = os.path.join(project_root, "src")

# Final unified paths
results_root = os.path.join(src_root, "results")
data_root = os.path.join(src_root, "data","MAMS-ACSA","raw","data_jsonl")
print(f"📂 Project root: {project_root}"
      f"\n📂 Source root: {src_root}"
      f"\n📂 Results root: {results_root}"
      f"\n📂 Data root: {data_root}")

📂 Project root: /Users/hd/Desktop/EMOTION-PRED
📂 Source root: /Users/hd/Desktop/EMOTION-PRED/src
📂 Results root: /Users/hd/Desktop/EMOTION-PRED/src/results
📂 Data root: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl


In [2]:
file = os.path.join(results_root,"emotion_MAMS-ACSA","joeddav_distilbert_base_uncased_go_emotions_student_annotated.jsonl")

In [3]:
import json

go_data = []

with open(file, "r", encoding="utf-8") as f:
    for line in f:
        go_data.append(json.loads(line))


In [4]:
go_data[0]


{'input': '$25 prix fixe sounds like a good deal, but dinner was still $100 for two, the portions were small, and the ($35) wine, clearly where the money is made, mediocre.',
 'output': [{'aspect': 'price', 'polarity': 'neutral', 'emotion': 'confusion'},
  {'aspect': 'miscellaneous', 'polarity': 'negative', 'emotion': 'confusion'},
  {'aspect': 'food', 'polarity': 'negative', 'emotion': 'confusion'}]}

In [5]:
aspect_counts = [len(entry["output"]) for entry in go_data]


from collections import Counter

aspect_count_distribution = Counter(aspect_counts)
aspect_count_distribution


Counter({2: 2452, 3: 609, 4: 81, 5: 7})

In [6]:
go_emotions_list = []

for entry in go_data:
    for item in entry["output"]:
        go_emotions_list.append(item["emotion"])


In [7]:
sorted(set(go_emotions_list))

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'joy',
 'love',
 'nervousness',
 'neutral',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise']

In [8]:
aspect_list = []
for entry in go_data:
    for item in entry["output"]:
        aspect_list.append(item["aspect"])

sorted(set(aspect_list))

['ambience',
 'food',
 'menu',
 'miscellaneous',
 'place',
 'price',
 'service',
 'staff']

In [9]:
from collections import Counter

go_counts = Counter(go_emotions_list)
go_counts_sorted = dict(sorted(go_counts.items()))
go_counts_sorted

{'admiration': 62,
 'amusement': 100,
 'anger': 93,
 'annoyance': 769,
 'approval': 75,
 'caring': 2578,
 'confusion': 686,
 'curiosity': 178,
 'desire': 345,
 'disappointment': 520,
 'disapproval': 122,
 'disgust': 8,
 'embarrassment': 50,
 'excitement': 373,
 'fear': 4,
 'gratitude': 15,
 'joy': 80,
 'love': 4,
 'nervousness': 107,
 'neutral': 45,
 'pride': 13,
 'realization': 494,
 'relief': 141,
 'remorse': 65,
 'sadness': 16,
 'surprise': 147}

In [10]:
pairs = []

for entry in go_data:
    for item in entry["output"]:
        emotion = item["emotion"]
        polarity = item["polarity"]
        pairs.append((emotion, polarity))

import pandas as pd
df_pairs = pd.DataFrame(pairs, columns=["emotion", "polarity"])
pivot = df_pairs.pivot_table(
    index="emotion",
    columns="polarity",
    aggfunc="size",
    fill_value=0
)
pivot["total"] = pivot.sum(axis=1)
pivot


polarity,negative,neutral,positive,total
emotion,,,,
admiration,10,21,31,62
amusement,29,40,31,100
anger,48,43,2,93
annoyance,370,316,83,769
approval,8,33,34,75
caring,474,1151,953,2578
confusion,268,330,88,686
curiosity,52,80,46,178
desire,89,153,103,345


In [11]:
pivot["neg_pct"] = pivot["negative"] / pivot["total"]
pivot["neu_pct"] = pivot["neutral"] / pivot["total"]
pivot["pos_pct"] = pivot["positive"] / pivot["total"]
pivot[["neg_pct", "neu_pct", "pos_pct"]].head(30)


polarity,neg_pct,neu_pct,pos_pct
emotion,,,
admiration,0.161290,0.338710,0.500000
amusement,0.290000,0.400000,0.310000
anger,0.516129,0.462366,0.021505
annoyance,0.481144,0.410923,0.107932
approval,0.106667,0.440000,0.453333
caring,0.183863,0.446470,0.369666
confusion,0.390671,0.481050,0.128280
curiosity,0.292135,0.449438,0.258427
desire,0.257971,0.443478,0.298551


In [12]:
pivot["purity"] = pivot[["neg_pct", "neu_pct", "pos_pct"]].max(axis=1)
pivot["purity"].sort_values(ascending=False).head()
pivot

# Basically least pure are those with near equal distribution across all three polarities

polarity,negative,neutral,positive,total,neg_pct,neu_pct,pos_pct,purity
emotion,,,,,,,,
admiration,10,21,31,62,0.161290,0.338710,0.500000,0.500000
amusement,29,40,31,100,0.290000,0.400000,0.310000,0.400000
anger,48,43,2,93,0.516129,0.462366,0.021505,0.516129
annoyance,370,316,83,769,0.481144,0.410923,0.107932,0.481144
approval,8,33,34,75,0.106667,0.440000,0.453333,0.453333
caring,474,1151,953,2578,0.183863,0.446470,0.369666,0.446470
confusion,268,330,88,686,0.390671,0.481050,0.128280,0.481050
curiosity,52,80,46,178,0.292135,0.449438,0.258427,0.449438
desire,89,153,103,345,0.257971,0.443478,0.298551,0.443478


In [13]:
pivot["purity"].sort_values(ascending=False).head()
# top 5 most pure emotions are just top five max values 

emotion
joy            0.550000
disapproval    0.540984
remorse        0.523077
anger          0.516129
nervousness    0.504673
Name: purity, dtype: float64

In [14]:
from sklearn.cluster import KMeans
import pandas as pd

X = pivot[["neg_pct", "neu_pct", "pos_pct"]].values
emotion_names = pivot.index.tolist()
emotion_names

kmeans = KMeans(n_clusters=8, random_state=42)
labels = kmeans.fit_predict(X)

pivot["cluster"] = labels
pivot[["neg_pct", "neu_pct", "pos_pct", "cluster"]]

cluster_groups = pivot.groupby("cluster").apply(lambda df: df.index.tolist())
cluster_groups



/var/folders/by/cnzpgkq568d37lzdkvmc0vsc0000gn/T/ipykernel_35459/2713176663.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_groups = pivot.groupby("cluster").apply(lambda df: df.index.tolist())


cluster
0    [annoyance, disappointment, disapproval, embar...
1                    [approval, gratitude, joy, pride]
2    [amusement, curiosity, desire, neutral, realiz...
3                           [admiration, love, relief]
4                                        [anger, fear]
5                        [caring, excitement, remorse]
6                                   [disgust, sadness]
7                                          [confusion]
dtype: object


# ⭐ **Cluster 0 — Mild Negative / Complaints**

```
[annoyance, disappointment, disapproval, embarrassment, nervousness, surprise]
```

### Meaning:

* “Something is wrong.”
* Complaints, frustration, discomfort.
* Not extreme negativity, but negative vibes.

---

# ⭐ **Cluster 1 — Clear Positive Feelings**

```
[approval, gratitude, joy, pride]
```

### Meaning:

* Happy, satisfied, thankful.
* Direct positive reactions.

---

# ⭐ **Cluster 2 — Neutral / Thinking / Mixed Emotions**

```
[amusement, curiosity, desire, neutral, realization, remorse]
```

### Meaning:

* Emotions that don’t point strongly to good or bad.
* Cognitive states (thinking, wanting, reflecting).
* Very mixed polarity → unclear sentiment.

---

# ⭐ **Cluster 3 — Warm Positive Emotions**

```
[admiration, love, relief]
```

### Meaning:

* Emotional warmth.
* “I feel good about you/this.”

---

# ⭐ **Cluster 4 — Strong Negative Emotions**

```
[anger, fear]
```

### Meaning:

* High intensity negativity.
* Threat, danger, rage.

---

# ⭐ **Cluster 5 — Soft Emotional Engagement**

```
[caring, excitement, remorse]
```

### Meaning:

* Personal involvement.
* Not clearly positive or negative.
* Emotional activation (care, excitement, regret).

---

# ⭐ **Cluster 6 — Pure Negative / Suffering**

```
[disgust, sadness]
```

### Meaning:

* Deep unpleasant feelings.
* Pain, rejection, emotional hurt.

---

# ⭐ **Cluster 7 — Confusion (Alone)**

```
[confusion]
```

### Meaning:

* Confusion behaves differently from everything else.
* Mixed polarity → unique pattern.
* KMeans isolates it.

---

# 🎯 **SUPER SIMPLE SUMMARY**

| Cluster | Meaning                    |
| ------- | -------------------------- |
| 0       | complaints / irritation    |
| 1       | happy feelings             |
| 2       | thinking / neutral / mixed |
| 3       | warm positive emotions     |
| 4       | strong negative reactions  |
| 5       | soft emotional involvement |
| 6       | deep negative suffering    |
| 7       | confusion alone            |

---

# 🧠 Why this is useful

* 28 emotions reduced to **8 clear groups**
* Each group has **similar polarity behavior**
* You now have **clean categories** for your emotion model
* Much more stable than using all 28 emotions


In [15]:
cluster_name_map = {
    0: "complaint",
    1: "positive_appreciation",
    2: "cognitive_neutral",
    3: "warm_positive",
    4: "intense_negative",
    5: "soft_emotional",
    6: "pure_negative",
    7: "confusion"
}

emotion_to_cluster = {}

for idx, row in pivot.iterrows():
    emotion = idx
    cluster_id = row["cluster"]
    emotion_to_cluster[emotion] = cluster_name_map[cluster_id]

emotion_to_cluster


{'admiration': 'warm_positive',
 'amusement': 'cognitive_neutral',
 'anger': 'intense_negative',
 'annoyance': 'complaint',
 'approval': 'positive_appreciation',
 'caring': 'soft_emotional',
 'confusion': 'confusion',
 'curiosity': 'cognitive_neutral',
 'desire': 'cognitive_neutral',
 'disappointment': 'complaint',
 'disapproval': 'complaint',
 'disgust': 'pure_negative',
 'embarrassment': 'complaint',
 'excitement': 'soft_emotional',
 'fear': 'intense_negative',
 'gratitude': 'positive_appreciation',
 'joy': 'positive_appreciation',
 'love': 'warm_positive',
 'nervousness': 'complaint',
 'neutral': 'cognitive_neutral',
 'pride': 'positive_appreciation',
 'realization': 'cognitive_neutral',
 'relief': 'warm_positive',
 'remorse': 'soft_emotional',
 'sadness': 'pure_negative',
 'surprise': 'cognitive_neutral'}

In [16]:
cluster_to_emotions = {
    "complaint": [
        "annoyance",
        "disappointment",
        "disapproval",
        "embarrassment",
        "nervousness"
    ],

    "positive_appreciation": [
        "approval",
        "gratitude",
        "joy",
        "pride"
    ],

    "cognitive_neutral": [
        "amusement",
        "curiosity",
        "desire",
        "neutral",
        "realization",
        "surprise"
    ],

    "warm_positive": [
        "admiration",
        "love",
        "relief"
    ],

    "intense_negative": [
        "anger",
        "fear"
    ],

    "soft_emotional": [
        "caring",
        "excitement",
        "remorse"
    ],

    "pure_negative": [
        "disgust",
        "sadness"
    ],

    "confusion": [
        "confusion"
    ]
}


In [17]:
import os
import json

# paths
data_root = os.path.join(src_root, "data", "MAMS-ACSA", "raw", "data_jsonl")
output_path = os.path.join(data_root, "train_clustered_emotion.jsonl")

# 1. Replace emotion with its cluster
for entry in go_data:
    for item in entry["output"]:
        original_emo = item["emotion"]
        cluster = emotion_to_cluster[original_emo]
        item["emotion"] = cluster  # overwrite original emotion

# 2. Write updated JSONL file
with open(output_path, "w", encoding="utf-8") as f:
    for entry in go_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print("Saved:", output_path)


Saved: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl/train_clustered_emotion.jsonl



### **1️⃣ WHAT:** Loaded the MAMS JSONL dataset.

**WHY:** Needed raw inputs + model-predicted emotions for analysis.
**HOW:** Read each JSONL line into Python and stored as dictionaries.

---

### **2️⃣ WHAT:** Extracted all emotions tied to each aspect.

**WHY:** To understand how emotions are distributed across the dataset.
**HOW:** Iterated through each entry and collected emotion labels into lists.

---

### **3️⃣ WHAT:** Counted emotions by polarity (neg/neu/pos).

**WHY:** Emotions behave differently depending on sentiment; we must quantify this.
**HOW:** Built a pivot table using negative/neutral/positive counts per emotion.

---

### **4️⃣ WHAT:** Computed polarity percentages and emotion “purity.”

**WHY:** To see which emotions strongly lean negative or positive, and which are mixed.
**HOW:** Divided each emotion’s polarity counts by the total number of occurrences.

---

### **5️⃣ WHAT:** Observed high noise and overlap across 28 emotions.

**WHY:** Many emotions don’t map cleanly to positive/negative, making them hard to classify.
**HOW:** Inspected purity results and frequency distributions.

---

### **6️⃣ WHAT:** Applied KMeans clustering to emotion polarity patterns.

**WHY:** To reduce 28 chaotic emotions into stable, interpretable groups.
**HOW:** Clustered emotions using their neg%, neu%, pos% as input features.

---

### **7️⃣ WHAT:** Interpreted 8 natural emotion clusters.

**WHY:** Clusters revealed meaningful emotional themes (frustration, joy-like, mixed, etc.).
**HOW:** Looked at which emotions fell together and assigned intuitive cluster names.

---

### **8️⃣ WHAT:** Created two mappings: cluster → emotions and emotion → cluster.

**WHY:** Needed a clean, error-free way to convert original emotions into our new 8 categories.
**HOW:** Wrote dictionaries for both directions based on clustering output.

---

### **9️⃣ WHAT:** Replaced each original emotion with its cluster label.

**WHY:** This simplifies the dataset and prepares it for model training.
**HOW:** Overwrote “emotion” field in every output object with the cluster name.

---

### **🔟 WHAT:** Saved the new dataset as `train_clustered_emotion.jsonl`.

**WHY:** To produce a clean training file for our model with consistent emotion labels.
**HOW:** Wrote each updated entry back into a new JSONL file in the data_root directory.

---

If you want a **super short 5-bullet version**, a **diagram**, or a **team-friendly slide summary**, just say **next**.
